# The Battle of the Neighborhoods Report

## Introduction/ Business Problem

My client is a small business owner from New York and is trying to open a second location of their business in the city of Toronto. The business is a family-owned Italian restaurant. In the past, my client's biggest struggle has been completing with other Italian restaurants. Therefore, I will recommend a location for my client based on the concentration of my client's competitors in a neighborhood. 

## Data and Method

I will be using Foursquare API Data to assess the Toronto. The data points that I will be using are the locations of Italian restaurants. Based on the concentration of these locations I will recommend a location for my client. My recommendation will most likely be in a location where there is a low concentration of these restaurants, but there are still some present. The low concentration is important because it means that the competition will be less. The presence of other Italian restaurants is also important in establishing my client's new business location because it means that the demographic of that neighborhood probably enjoys Italian food and will become patrons of my client. 

#### Import Libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [4]:
from geopy.geocoders import Nominatim 

import matplotlib.cm as cm
import matplotlib.colors as colors
 
from sklearn.cluster import KMeans
import folium 

#### Define Foursquare Credentials 

In [52]:
CLIENT_ID = 'EZJRMM13FKSKWYDHDPRC23MLN2QYZQT4BAAFP3E2KDJG2DI5' # your Foursquare ID
CLIENT_SECRET = 'LQF4QP2QVLS5KL1D53GXPQT3OGHTTRTH4XWLYKKFHUALCDBO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EZJRMM13FKSKWYDHDPRC23MLN2QYZQT4BAAFP3E2KDJG2DI5
CLIENT_SECRET:LQF4QP2QVLS5KL1D53GXPQT3OGHTTRTH4XWLYKKFHUALCDBO


#### Search and Clean Dataframe

In [70]:
search_query = 'Italian'
radius = 10000
print(search_query + ' .... OK!')

Italian .... OK!


In [71]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


In [72]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, 43.653963, -79.387207, VERSION, search_query, radius, LIMIT)

In [73]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d4c584cdb1d81002cec6c5d'},
 'response': {'venues': [{'id': '52f6816f11d24a43115dc834',
    'name': 'Scaddabush Italian Kitchen & Bar',
    'location': {'address': '382 Yonge Street, Unit #7',
     'crossStreet': 'Gerrard',
     'lat': 43.658920292028725,
     'lng': -79.38289105381784,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.658920292028725,
       'lng': -79.38289105381784}],
     'distance': 652,
     'postalCode': 'M5B 1S8',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['382 Yonge Street, Unit #7 (Gerrard)',
      'Toronto ON M5B 1S8',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'shortName': 'Italian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId

In [74]:
venues = results['response']['venues'] 

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,52f6816f11d24a43115dc834,"382 Yonge Street, Unit #7",CA,Toronto,Canada,Gerrard,652,"[382 Yonge Street, Unit #7 (Gerrard), Toronto ...","[{'label': 'display', 'lat': 43.65892029202872...",43.658920,-79.382891,M5B 1S8,ON,Scaddabush Italian Kitchen & Bar,v-1565284428,NaN
1,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,5b897e92db1d81002c91df8c,66 Wellington St W,CA,Toronto,Canada,NaN,877,"[66 Wellington St W, Toronto ON M5K 1E7, Canada]","[{'label': 'display', 'lat': 43.647161, 'lng':...",43.647161,-79.381691,M5K 1E7,ON,Fabbrica Rustic Italian,v-1565284428,NaN
2,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",False,4e31afdd091a973ec9c5a2b5,146 Cumberland St,CA,Toronto,Canada,btwn Avenue Rd & Bay St,1832,"[146 Cumberland St (btwn Avenue Rd & Bay St), ...","[{'label': 'display', 'lat': 43.66995452843031...",43.669955,-79.392603,M5R 1A8,ON,"Punto Gelato, Simply Italian",v-1565284428,NaN
3,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,581cad6a7c74e15859a6f890,"200 Front Street W., Unit #G001",CA,Toronto,Canada,at Simcoe St,1037,"[200 Front Street W., Unit #G001 (at Simcoe St...","[{'label': 'display', 'lat': 43.6447367776608,...",43.644737,-79.385355,M5V 3J1,ON,Scaddabush Italian Kitchen & Bar,v-1565284428,NaN
4,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",False,4bfc0289c3ba9521c00f9653,136 Beverley St,CA,Toronto,Canada,Dundas Street,555,"[136 Beverley St (Dundas Street), Toronto ON, ...","[{'label': 'display', 'lat': 43.65402694219784...",43.654027,-79.394104,NaN,ON,Italian Consulate Toronto,v-1565284428,NaN


In [81]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Scaddabush Italian Kitchen & Bar,Italian Restaurant,"382 Yonge Street, Unit #7",CA,Toronto,Canada,Gerrard,652,"[382 Yonge Street, Unit #7 (Gerrard), Toronto ...","[{'label': 'display', 'lat': 43.65892029202872...",43.658920,-79.382891,M5B 1S8,ON,52f6816f11d24a43115dc834
1,Fabbrica Rustic Italian,Italian Restaurant,66 Wellington St W,CA,Toronto,Canada,NaN,877,"[66 Wellington St W, Toronto ON M5K 1E7, Canada]","[{'label': 'display', 'lat': 43.647161, 'lng':...",43.647161,-79.381691,M5K 1E7,ON,5b897e92db1d81002c91df8c
2,"Punto Gelato, Simply Italian",Ice Cream Shop,146 Cumberland St,CA,Toronto,Canada,btwn Avenue Rd & Bay St,1832,"[146 Cumberland St (btwn Avenue Rd & Bay St), ...","[{'label': 'display', 'lat': 43.66995452843031...",43.669955,-79.392603,M5R 1A8,ON,4e31afdd091a973ec9c5a2b5
3,Scaddabush Italian Kitchen & Bar,Italian Restaurant,"200 Front Street W., Unit #G001",CA,Toronto,Canada,at Simcoe St,1037,"[200 Front Street W., Unit #G001 (at Simcoe St...","[{'label': 'display', 'lat': 43.6447367776608,...",43.644737,-79.385355,M5V 3J1,ON,581cad6a7c74e15859a6f890
4,Italian Consulate Toronto,Embassy / Consulate,136 Beverley St,CA,Toronto,Canada,Dundas Street,555,"[136 Beverley St (Dundas Street), Toronto ON, ...","[{'label': 'display', 'lat': 43.65402694219784...",43.654027,-79.394104,NaN,ON,4bfc0289c3ba9521c00f9653
5,The Fresh Italian,Italian Restaurant,NaN,CA,Toronto,Canada,NaN,127,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65499143746528...",43.654991,-79.387897,NaN,ON,51bf3866498e55ee55df8db0
6,Jamie's Italian,Italian Restaurant,3401 Dufferin St,CA,Toronto,Canada,Allen Road and 401,9679,"[3401 Dufferin St (Allen Road and 401), Toront...","[{'label': 'display', 'lat': 43.72668644119483...",43.726686,-79.453133,M6A 2T9,ON,566344c9498eedf4e11af0fa
7,Kit Kat Italian Bar & Grill,Italian Restaurant,297 King St W,CA,Toronto,Canada,at John St,870,"[297 King St W (at John St), Toronto ON M5V 1J...","[{'label': 'display', 'lat': 43.64641598988062...",43.646416,-79.390030,M5V 1J5,ON,4b3ace79f964a520ae6e25e3
8,The Fresh Italian Eatery,Italian Restaurant,"109 McCaul Street, Unit #42",CA,Toronto,Canada,Dundas Street West,288,"[109 McCaul Street, Unit #42 (Dundas Street We...","[{'label': 'display', 'lat': 43.653889, 'lng':...",43.653889,-79.390785,M5T 3K5,ON,526fe29411d2aeb3803013b0
9,Mustachio Italian Eatery,Italian Restaurant,595 Bay St,CA,Toronto,Canada,Dundas St,405,"[595 Bay St (Dundas St), Toronto ON M5G 2C2, C...","[{'label': 'display', 'lat': 43.65616, 'lng': ...",43.656160,-79.383190,M5G 2C2,ON,573df789498e03dd8e54b166


In [84]:
dataframe_filtered.drop('Voting Booth', 'Historic Site', 'Ice Cream Shop', 'Conference Room', 'Art Gallery', 'Embassy/ Consulate'], axis = 0)

KeyError: "['Voting Booth' 'Historic Site' 'Ice Cream Shop' 'Conference Room'\n 'Art Gallery' 'Embassy/ Consulate'] not found in axis"

#### Visualize Data 

In [68]:
dataframe_filtered.name

0                 Italian Consulate Toronto
1                         The Fresh Italian
2                  The Fresh Italian Eatery
3                  Mustachio Italian Eatery
4                        LA's Italian + Bar
5                       john's italian cafe
6    Classic italian style pizza food truck
7                          Little Anthony's
8                                    Sbarro
Name: name, dtype: object

In [76]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Toronto City Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

#### Perform a K-Means test to determine the region with the least concentration of Italian Restaurants 

In [ ]:
kclus = 5

df_cluster = dfC.drop('Neighbourhood', 1)
df_cluster = dfC_cluster.drop('Borough', 1)
df_cluster = dfC_cluster.drop('Postcode', 1)

kmeans = KMeans(n_clusters = kclus, random_state = 0).fit(df_cluster)

kmeans.labels_[1:10]